In [1]:
import pandas as pd
import duckdb

# load dataset
df = pd.read_csv("/kaggle/input/customer-churn-prediction-dataset/customer_churn_dataset.csv")

# SQL connection
con = duckdb.connect()
con.register("customers", df)

df.head()

,customer_id,tenure,monthly_charges,total_charges,contract,payment_method,internet_service,tech_support,online_security,support_calls,churn
0,1,52,54.20,2818.40,Month-to-month,Credit,DSL,No,Yes,1,No
1,2,15,35.28,529.20,Month-to-month,Debit,DSL,No,No,2,No
2,3,72,78.24,5633.28,Month-to-month,Debit,DSL,No,No,0,No
3,4,61,80.24,4894.64,One year,Cash,Fiber,Yes,Yes,0,No
4,5,21,39.38,826.98,Month-to-month,UPI,Fiber,No,No,4,Yes


In [2]:
con.execute("""
SELECT COUNT(*) AS total_customers
FROM customers
""").df()


,total_customers
0,20000


In [3]:
con.execute("""
SELECT COUNT(*) AS churned_customers
FROM customers
WHERE churn = 'Yes'
""").df()



,churned_customers
0,6843


In [4]:
con.execute("""
SELECT 
    COUNT(*) * 100.0 / (SELECT COUNT(*) FROM customers) AS churn_rate_pct
FROM customers
WHERE churn = 'Yes'
""").df()

,churn_rate_pct
0,34.215


In [5]:
con.execute("""
SELECT
    contract,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(
        SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS churn_rate_pct
FROM customers
GROUP BY contract
ORDER BY churn_rate_pct DESC
""").df()


,contract,total_customers,churned_customers,churn_rate_pct
0,Month-to-month,11942,5157.0,43.18
1,Two year,3068,642.0,20.93
2,One year,4990,1044.0,20.92


Customers on month-to-month contracts have a significantly higher churn rate (43%) compared to long-term contracts

In [6]:
con.execute("""
SELECT
    -- Group customers by tenure range (months)
    CASE
        WHEN tenure < 12 THEN '0–11 months'
        WHEN tenure BETWEEN 12 AND 24 THEN '12–24 months'
        WHEN tenure BETWEEN 25 AND 48 THEN '25–48 months'
        ELSE '49+ months'
    END AS tenure_group,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(
        SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS churn_rate_pct
FROM customers
GROUP BY tenure_group
ORDER BY churn_rate_pct DESC
""").df()


,tenure_group,total_customers,churned_customers,churn_rate_pct
0,0–11 months,2995,1425.0,47.58
1,12–24 months,3744,1213.0,32.40
2,25–48 months,6562,2112.0,32.19
3,49+ months,6699,2093.0,31.24


Customers in their first year show the highest churn rate (47.6%), with churn decreasing as tenure increases.

In [7]:
con.execute("""
SELECT
      -- Group customers by number of support calls
    CASE
        WHEN support_calls = 0 THEN '0 calls'
        WHEN support_calls BETWEEN 1 AND 2 THEN '1–2 calls'
        WHEN support_calls BETWEEN 3 AND 4 THEN '3–4 calls'
        ELSE '5+ calls'
    END AS support_calls_group,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(
        SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS churn_rate_pct
FROM customers
GROUP BY support_calls_group
ORDER BY churn_rate_pct DESC
""").df()


,support_calls_group,total_customers,churned_customers,churn_rate_pct
0,5+ calls,424,256.0,60.38
1,3–4 calls,3481,2024.0,58.14
2,0 calls,4448,1263.0,28.39
3,1–2 calls,11647,3300.0,28.33


Customers with a higher number of support calls show significantly higher churn rates, exceeding 58% among customers with three or more calls.

In [8]:
con.execute("""
WITH base AS (
    SELECT
        contract,
        -- Group customers by number of support calls
        CASE
            WHEN support_calls = 0 THEN '0 calls'
            WHEN support_calls BETWEEN 1 AND 2 THEN '1–2 calls'
            WHEN support_calls BETWEEN 3 AND 4 THEN '3–4 calls'
            ELSE '5+ calls'
        END AS support_calls_group,
        churn
    FROM customers
)
SELECT
    contract,
    support_calls_group,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(
        SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS churn_rate_pct
FROM base
GROUP BY contract, support_calls_group
ORDER BY churn_rate_pct DESC
""").df()


,contract,support_calls_group,total_customers,churned_customers,churn_rate_pct
0,Month-to-month,3–4 calls,2099,1523.0,72.56
1,Month-to-month,5+ calls,266,190.0,71.43
2,One year,5+ calls,99,42.0,42.42
3,Two year,5+ calls,59,24.0,40.68
4,One year,3–4 calls,850,323.0,38.00
5,Month-to-month,1–2 calls,6899,2496.0,36.18
6,Month-to-month,0 calls,2678,948.0,35.40
7,Two year,3–4 calls,532,178.0,33.46
8,Two year,0 calls,682,127.0,18.62
9,Two year,1–2 calls,1795,313.0,17.44


**Final Conclusions**

The overall churn rate is high (34%), indicating a significant customer retention challenge.

Customers on month-to-month contracts consistently show higher churn rates compared to long-term contracts.

Churn is highest during the first year of the customer lifecycle, especially among customers with less than 12 months of tenure.

The highest-risk segment consists of month-to-month customers with multiple support calls, where churn exceeds 70%.

Churn is highest among month-to-month customers with multiple support calls, exceeding 70%, indicating a high-risk segment.